In [1]:
#One time run to import the project to your drive
!git clone https://github.com/KodeWil/final_project_AI_csv_Data.git

Cloning into 'final_project_AI_csv_Data'...
remote: Enumerating objects: 211, done.
remote: Counting objects: 100% (211/211), done.
remote: Compressing objects: 100% (209/209), done.
remote: Total 211 (delta 2), reused 211 (delta 2), pack-reused 0
Receiving objects: 100% (211/211), 17.32 MiB | 14.12 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [0]:
def addData(files: str) -> list:
    allCsv = []
    for file in files:
      temp = open(file)
      tempDf = pd.read_csv(file)
      allCsv.append(tempDf)
    return allCsv


In [0]:
import pandas as pd 
import glob 
frames = []

#Create a list ('frames') with four nested lists, one per sensor. 
#Each nested list have all the samples (DF) for that sensor.
%cd /content/final_project_AI_csv_Data/phone/accel
files = glob.glob('*.csv')
files.sort()
frames.append(addData(files))
%cd /content/final_project_AI_csv_Data/phone/gyro
files = glob.glob('*.csv')
files.sort()
frames.append(addData(files))
%cd /content/final_project_AI_csv_Data/watch/accel
files = glob.glob('*.csv')
files.sort()
frames.append(addData(files))
%cd /content/final_project_AI_csv_Data/watch/gyro
files = glob.glob('*.csv')
files.sort()
frames.append(addData(files))
print(len(frames))
# 0 in frames = /phone/accel
# 1 in frames = /phone/gyro
# 2 in frames = /watch/accel
# 3 in frames = /watch/gyro

In [0]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
import matplotlib.pyplot as plt
import copy
from sklearn.manifold import LocallyLinearEmbedding
from sklearn.manifold import MDS

In [0]:
def colorMap(difClass: list) -> list: #Funcion to color map the classes.
    cMap = ["#ff3333", "#ff7b33", "#ffc933", "#89ff33", "#33ff49", 
           "#33fff2", "#33aaff", "#335cff", "#5133ff", "#a333ff",
           "#f133ff", "#e483b1", "#979797", "#752f7e", "#6d067b",
           "#ff4dc7", "#c900bd", "#63002c"]
    cList = []
    for i, values in enumerate(difClass):
      color = cMap[difClass[i]]
      cList.append(color)
    return cList

In [5]:
# preparamos los datos para el clustering
d_accel = frames[0][1]
#d_accel = datos_accel
print(d_accel.shape)

(407, 93)


In [0]:
#Directly related features are wrapped in intervals on a list
def selectFeatures(features) -> list:
  labels = copy.deepcopy(features)
  intervals = []
  intervals.append(labels[1:30]) 
  intervals.append(labels[31:33]) # {X,Y,Z}AVG 
  intervals.append(labels[34:36]) # {X,Y,Z}PEAK 
  intervals.append(labels[37:39]) # {X,Y,Z}ABSOLDEV 
  intervals.append(labels[40:42]) # {X,Y,Z}STANDDEV 
  intervals.append(labels[43:45]) # {X,Y,Z}VAR* 
  intervals.append(labels[46:84]) # FCC
  intervals.append(labels[85:87]) # COS
  intervals.append(labels[88:90]) # COR
  # intervals.append(labels[91:92]) # resultant
  return intervals


In [0]:
def allPca(X):
  pca = PCA(n_components=2,whiten=False).fit(X)
  # print(pca.explained_variance_ratio_)
  # print(pca.singular_values_)
  transformados = pca.fit_transform(X)
  # print(transformados.shape)
  allPlot(transformados)

In [0]:
def allIca(X):
  ica = FastICA(n_components=2,whiten=True).fit(X)
  # print(pca.explained_variance_ratio_)
  # print(pca.singular_values_)
  transformados = ica.fit_transform(X)
  # print(transformados.shape)
  allPlot(transformados)

In [0]:
def allLLE(X):
  lle = LocallyLinearEmbedding(n_neighbors=201,n_components=2).fit(X)
  transformados = lle.transform(X)
  allPlot(transformados)

In [0]:
def allMDS(X, metrica):
  mds = MDS(n_components = 2,metric = metrica).fit(X)
  transformados = mds.fit_transform(X)
  allPlot(transformados)

In [0]:
def allPlot(transformados):
  x = transformados[:,0]
  y = transformados[:,1]
  c = ground_truth
  # c = colorMap(c)
  fig, ax = plt.subplots(figsize=[10,10])
  scatter = ax.scatter(x, y, c = c)
  # produce a legend with the unique colors from the scatter
  legend1 = ax.legend(*scatter.legend_elements(),
                      loc = 1, title="Classes")
  ax.add_artist(legend1)

  plt.show()

In [0]:
ground_truth = d_accel[["ACTIVITY"]].values
lookupTable, ground_truth = np.unique(ground_truth, return_inverse=True)
columnas = list(d_accel.columns[1:92])
wrapInf = selectFeatures(columnas)

#PCA and ICA in that order
for piece in wrapInf:
  print("PCA and ICA for the features: ")
  print(piece)
  print("")
  X = d_accel[piece].values
  print("PCA ")
  print("")
  allPca(X)
  print("ICA")
  print("")
  allIca(X)
  print("")

#LLE and MDS (Non metric and Metric in that order)
for piece in wrapInf:
  print("LLE, MDS (non metric and  metric) for the features: ")
  print(piece)
  print("")
  X = d_accel[piece].values
  print("LLE")
  print("")
  allLLE(X)
  print("MDS non metric")
  print("")
  allMDS(X, False)
  print("MDS metric")
  print("")
  allMDS(X, True)



